In [2]:
import nltk
nltk.download('conll2000')

[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


True

In [11]:
from nltk.corpus import conll2000
cp= nltk.RegexpParser("")
test_sents=conll2000.chunked_sents('test.txt',chunk_types=['NP'])
print(cp.accuracy(test_sents))

ChunkParse score:
    IOB Accuracy:  43.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


In [12]:
grammar=r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammar)
print(cp.accuracy(test_sents))

ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     70.6%%
    Recall:        67.8%%
    F-Measure:     69.2%%


In [13]:
# Unigram taggers
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self,train_sents):
        train_data=[[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)
    def parse(self,sentence):
        pos_tags=[pos for (word,pos) in sentence]
        tagged_pos_tags=self.tagger.tag(pos_tags)
        chunktags=[chunktag for (pos,chunktag) in tagged_pos_tags]
        conlltags=[(word,pos,chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [14]:
test_sents=conll2000.chunked_sents('test.txt',chunk_types=['NP'])
train_sents=conll2000.chunked_sents('train.txt',chunk_types=['NP'])
unigram_chunker=UnigramChunker(train_sents)
print(unigram_chunker.accuracy(test_sents))

ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%


In [15]:
postags=sorted(set(pos for sent  in train_sents for (word,pos) in sent.leaves()))
print(unigram_chunker.tagger.tag(postags))

[('#', 'B-NP'), ('$', 'B-NP'), ("''", 'O'), ('(', 'O'), (')', 'O'), (',', 'O'), ('.', 'O'), (':', 'O'), ('CC', 'O'), ('CD', 'I-NP'), ('DT', 'B-NP'), ('EX', 'B-NP'), ('FW', 'I-NP'), ('IN', 'O'), ('JJ', 'I-NP'), ('JJR', 'B-NP'), ('JJS', 'I-NP'), ('MD', 'O'), ('NN', 'I-NP'), ('NNP', 'I-NP'), ('NNPS', 'I-NP'), ('NNS', 'I-NP'), ('PDT', 'B-NP'), ('POS', 'B-NP'), ('PRP', 'B-NP'), ('PRP$', 'B-NP'), ('RB', 'O'), ('RBR', 'O'), ('RBS', 'B-NP'), ('RP', 'O'), ('SYM', 'O'), ('TO', 'O'), ('UH', 'O'), ('VB', 'O'), ('VBD', 'O'), ('VBG', 'O'), ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'B-NP'), ('WP', 'B-NP'), ('WP$', 'B-NP'), ('WRB', 'O'), ('``', 'O')]


In [16]:
class BigramChunker(nltk.ChunkParserI):
    def __init__(self,train_sents):
        train_data=[[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)
    def parse(self,sentence):
        pos_tags=[pos for (word,pos) in sentence]
        tagged_pos_tags=self.tagger.tag(pos_tags)
        chunktags=[chunktag for (pos,chunktag) in tagged_pos_tags]
        conlltags=[(word,pos,chunktag) for ((word,pos),chunktag) in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)


test_sents=conll2000.chunked_sents('test.txt',chunk_types=['NP'])
train_sents=conll2000.chunked_sents('train.txt',chunk_types=['NP'])
bigram_chunker=BigramChunker(train_sents)
print(bigram_chunker.accuracy(test_sents))

ChunkParse score:
    IOB Accuracy:  93.3%%
    Precision:     82.3%%
    Recall:        86.8%%
    F-Measure:     84.5%%


In [20]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):
    def __init__(self,train_sents):
        train_set=[]
        for tagged_sent in train_sents:
            untagged_sent=nltk.tag.untag(tagged_sent)
            history=[]
            for i, (word,tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent,i,history)
                train_set.append((featureset,tag))
                history.append(tag)
        self.classifier=nltk.MaxentClassifier.train(train_set,algorithm='megam',trace=0)
    def tag(self,sentence):
        history=[]
        for i , word in enumerate(sentence):
            featureset = npchunk_features(sentence,i,history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence,history)

In [25]:
class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self,train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in nltk.chunk.tree2conlltags(sent)] for sent in trian_sents]
        self.tagger= ConsecutiveNPChunkTagger(tagged_sents)
    def parse(self,sentence):
        tagged_sents=self.tagger.tag(sentence)
        conlltags=[(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [27]:
def npchunk_features(sentence,i,history):
  word,pos=sentence[i]
  return {'pos' : pos}
chunker = ConsecutiveNPChunker(train_sents)
print(chunker.accuracy(test_sents))

NameError: name 'trian_sents' is not defined